In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Test d'une routine sur la méthode de Galerkine
"""

DataFolder="C:/Users/Julie_000/Desktop/Stage/Stage_l3"

# Importations des librairies

In [2]:
from mat4py import loadmat #pour charger des .mat
import matplotlib.pyplot as plt # pour tracer les figures
import os # pour modifier le dossier de travail
import numpy as np 
from pathlib import Path # Pour rendre les Path compatibles entre Mac et Windows
from matplotlib import rc #Les trois prochaines lignes pour que Ã§a ressemble Ã  latex
rc('font', size=16)
rc('text', usetex=True)
import scipy as sc
import matplotlib.cm as cm #colormaps
from matplotlib.colors import Normalize #Pour l'utilisation des couleurs dans quiver
from matplotlib.ticker import MaxNLocator #pour les courbes de niveau
from scipy.ndimage import gaussian_filter #Filtrage gaussien

from scipy.linalg import pinv as scpinv
from numpy.linalg import pinv as nppinv

import fonctions_projet_divergence as fpd#pour les données

colormap2=cm.PiYG#colormap

# Chargement des données
## Définition des fonctions

In [3]:
def PIV(prof,manips): 
    """retourne les données des plans pour la bonne profondeur"""
    if prof==5:
        piv=manips['piv5']
    elif prof==10:
        piv=manips['piv10']
    elif prof==15:
        piv=manips['piv15']
    else:
        return("prof n'a pas une valeur acceptable")   
    return(piv)

def nb_plan(prof):
    "Détermine le nombre de plan pour une profondeur donnée"
    if prof==5:
        return(16)
    if prof==10:
        return(32)
    if prof==15:
        return(49)
    else:
        return("prof n'a pas une valeur acceptable")
    
def donnees(n,piv):
    "Charge les données pour la profondeur donnée"
    u=np.zeros((n,60,60))
    v=np.zeros((n,60,60))
    x=np.zeros((n,60,60))
    y=np.zeros((n,60,60))
    z=np.zeros((n))

    for plan in range(n): 
            u[plan]=np.array(piv[plan]['u'])
            v[plan]=np.array(piv[plan]['v'])
            x[plan]=np.array(piv[plan]['x'])
            y[plan]=np.array(piv[plan]['y'])
            z[plan]=np.array(piv[plan]['prof'])
    return(u,v,x,y,z)

## Choix de la profondeur de travail

In [4]:
prof=5 # Choisi sur quelle profondeur d'eau travailler (5, 10 ou 15 mm)

## Chargement des données

In [16]:
os.chdir(Path(DataFolder)) # se place dans le dossier où est rangée la manip
manips=loadmat('PIVnagseul.mat') # Charge les données dans la variable manip

piv=PIV(prof,manips)#données pour la profondeur

P=nb_plan(prof)#nombre de plan dans la profondeur choisie

u,v,x,y,z=donnees(P,piv)
print(np.shape(u),np.shape(v),np.shape(x),np.shape(y),np.shape(z))

(16, 60, 60) (16, 60, 60) (16, 60, 60) (16, 60, 60) (16,)


# Application du programme de divergence bidimensionnelle
## Fonction de plot

In [6]:
def plot_divergence(div,dx,dy,title):
    "Tracé des divergences"
    #norme
    vmax,vmin=np.max(div),np.min(div)
    norm=Normalize(vmin=vmin, vmax=vmax)
    levels = MaxNLocator(nbins=25).tick_values(vmin,vmax)
    
    fig, l0 = plt.subplots(num=title)
    im0 = l0.contourf(dx,dy,div,levels=levels, cmap=colormap2,norm=norm)
    fig.colorbar(im0,ax=l0)
   
    fig.tight_layout(pad = 3)
    fig.suptitle(title,fontsize=20)
    plt.show()

## Constitution du tableau de divergence bidimensionnelle

In [39]:
div_2D=np.zeros((P,59,59))
dx=np.zeros((P,59,59))
dy=np.zeros((P,59,59))

for plan in range(P):
    div_2D[plan],dx[plan],dy[plan]=fpd.divergence2D_gauss(u[plan],v[plan],x[plan],y[plan],1)
print(np.shape(div_2D))

(16, 59, 59)


## Plot

In [8]:
#%matplotlib notebook
#plan=2
#plot_divergence(div_2D[plan],dx[plan],dy[plan],"Tracé de la divergence")

# Application de la méthode de Galerkine
## Choix du nombre de mode

In [11]:
m=5

## Construction des différents tableaux
### Tableau des z

In [19]:
#z=np.reshape(z,(P,1))
print(z)
print(np.shape(z))

[0.5 0.8 1.1 1.4 1.7 2.  2.3 2.6 2.9 3.2 3.5 3.8 4.1 4.4 4.7 5. ]
(16,)


### Matrice M

In [27]:
h=prof
M=np.transpose(np.array([n*np.pi/h*np.cos(n*np.pi/h*z) for n in range(1,m+1)]))

pinv=nppinv(M)
print(np.shape(pinv))

(5, 16)
118


# Résolution
## Coefficients

In [46]:
div_2D=np.reshape(div_2D,(P,59*59)) #Est-ce que ça marche vraiment ?? Sinon le produit matriciel ne fonctionne pas..

In [62]:
a=np.matmul(pinv,div_2D)
#a=np.reshape(a,(5,59,59))
print(np.shape(a))

(5, 3481)


## Détermination de v_z

In [67]:
sin=np.transpose(np.array([np.sin(m*np.pi*z/h) for m in range(1,m+1)]))
print(np.shape(sin))
vz=np.matmul(sin,a)
vz=np.reshape(vz,(P,59,59))
print(np.shape(vz))

(16, 5)
(16, 59, 59)


# Fonction Galerkine

In [69]:
def methode_Galerkine(u,v,x,y,z,P,m,h):
    "Applique la méthode Galerkine"
    #divergence bidimensionelle
    div_2D=np.zeros((P,59,59))
    dx=np.zeros((P,59,59))
    dy=np.zeros((P,59,59))
    for plan in range(P):
        div_2D[plan],dx[plan],dy[plan]=fpd.divergence2D_gauss(u[plan],v[plan],x[plan],y[plan],1)
    #Matrice M
    M=nppinv(np.transpose(np.array([n*np.pi/h*np.cos(n*np.pi/h*z) for n in range(1,m+1)])))
    #Calcul des coefficients
    div_2D=np.reshape(div_2D,(P,59*59))
    a=np.matmul(pinv,div_2D)
    #calcul de vz
    sin=np.transpose(np.array([np.sin(m*np.pi*z/h) for m in range(1,m+1)]))
    vz=np.matmul(sin,a)
    vz=np.reshape(vz,(P,59,59))
    return(vz)